# Ejercicios Prácticos de Pytest para Data Engineering

En este notebook, encontrarás una serie de ejercicios prácticos para aplicar lo que has aprendido sobre pytest en el contexto de Data Engineering. Estos ejercicios están diseñados para reforzar los conceptos y técnicas presentados en los notebooks anteriores.

Utilizaremos el dataset de ventas de productos que hemos estado usando a lo largo del tutorial. Cada ejercicio incluye instrucciones detalladas y, en algunos casos, código inicial para ayudarte a comenzar.

## Configuración Inicial

Primero, vamos a importar las bibliotecas necesarias y cargar nuestro dataset:

In [1]:
import pandas as pd
import numpy as np
import pytest
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns

# Añadimos el directorio raíz al path para poder importar los módulos
sys.path.append(os.path.abspath('..'))

# Cargamos el dataset
df_ventas = pd.read_csv('../data/ventas_productos.csv')

# Mostramos las primeras filas
df_ventas.head()

,id,fecha,producto,categoria,precio,cantidad,descuento,total
0,1,2023-01-05,Laptop HP,Electrónica,899.99,1,0.05,854.99
1,2,2023-01-10,Monitor Dell,Electrónica,249.99,2,0.00,499.98
2,3,2023-01-15,Teclado Logitech,Accesorios,59.99,3,0.10,161.97
3,4,2023-01-20,Mouse Inalámbrico,Accesorios,29.99,5,0.00,149.95
4,5,2023-01-25,Disco SSD 500GB,Almacenamiento,89.99,2,0.15,152.98


## Ejercicio 1: Testing de una Función de Análisis de Rentabilidad

### Descripción

En este ejercicio, implementarás una función que calcule la rentabilidad de cada producto en el dataset de ventas y escribirás tests para verificar su correcto funcionamiento.

La rentabilidad se define como: `(precio * cantidad * (1 - descuento)) / (precio * cantidad) * 100`

Es decir, el porcentaje del ingreso potencial que realmente se obtuvo después de aplicar descuentos.

### Tarea 1: Implementa la función `calcular_rentabilidad`

Completa la siguiente función:

In [ ]:
def calcular_rentabilidad(df):
    """Calcula la rentabilidad de cada producto en el dataset de ventas.
    
    Args:
        df: DataFrame con columnas 'precio', 'cantidad', 'descuento' y 'total'
        
    Returns:
        DataFrame: DataFrame original con una columna adicional 'rentabilidad' (en porcentaje)
    """
    # Tu código aquí
    # Recuerda: rentabilidad = (precio * cantidad * (1 - descuento)) / (precio * cantidad) * 100
    
    # Sugerencia: Crea una copia del DataFrame para no modificar el original
    df_resultado = df.copy()
    
    # Calcula la rentabilidad
    # ...
    
    return df_resultado

In [2]:
def calcular_rentabilidad(df):
    
    # Crear una copia del DataFrame original
    df_resultado = df.copy()
    
    # Calcular el ingreso potencial (sin descuento)
    ingreso_potencial = df_resultado['precio'] * df_resultado['cantidad']
    
    # Calcular el ingreso real (con descuento)
    ingreso_real = ingreso_potencial * (1 - df_resultado['descuento'])
    
    # Calcular la rentabilidad en porcentaje
    df_resultado['rentabilidad'] = (ingreso_real / ingreso_potencial) * 100

    return df_resultado


### Tarea 2: Escribe tests para la función `calcular_rentabilidad`

Escribe al menos tres tests para verificar que la función `calcular_rentabilidad` funcione correctamente. Deberías verificar:

1. Que la función añada la columna 'rentabilidad' al DataFrame
2. Que los valores de rentabilidad sean correctos para algunos casos específicos
3. Que la función maneje correctamente casos especiales (por ejemplo, descuento = 0)

Completa el siguiente archivo de test:

In [3]:
%%file test_rentabilidad.py
import pytest
import pandas as pd
import numpy as np

def calcular_rentabilidad(df):
    """Calcula la rentabilidad de cada producto en el dataset de ventas."""
    df_resultado = df.copy()
    df_resultado['rentabilidad'] = (1 - df_resultado['descuento']) * 100
    return df_resultado

@pytest.fixture
def df_test():
    """Fixture que crea un DataFrame de prueba."""
    data = {
        'id': [1, 2, 3],
        'producto': ['Producto A', 'Producto B', 'Producto C'],
        'precio': [100.0, 200.0, 300.0],
        'cantidad': [2, 1, 3],
        'descuento': [0.1, 0.0, 0.25],
        'total': [180.0, 200.0, 675.0]
    }
    return pd.DataFrame(data)

def test_columna_rentabilidad_existe(df_test):
    resultado = calcular_rentabilidad(df_test)
    assert 'rentabilidad' in resultado.columns, "La columna 'rentabilidad' no fue añadida"

def test_valores_rentabilidad_correctos(df_test):
    resultado = calcular_rentabilidad(df_test)
    valores_esperados = [90.0, 100.0, 75.0]  # (1 - descuento) * 100
    np.testing.assert_allclose(resultado['rentabilidad'], valores_esperados, rtol=1e-2) #Compara que los valores reales de resultado['rentabilidad'] estén muy cerca (con tolerancia relativa del 1%) de los valores esperados

def test_caso_descuento_cero(df_test):
    resultado = calcular_rentabilidad(df_test)
    rentabilidad_producto_b = resultado.loc[resultado['producto'] == 'Producto B', 'rentabilidad'].values[0] # Filtra solo el producto "Producto B", obtiene el valor de su rentabilidad.
    assert rentabilidad_producto_b == 100.0, "Rentabilidad debería ser 100% cuando el descuento es 0" #Verifica que ese valor sea exactamente 100%, como se espera si no se aplicó ningún descuento.


Writing test_rentabilidad.py


poetry run pytest test_rentabilidad.py esto en consola

### Tarea 3: Ejecuta los tests

Ejecuta los tests que has escrito para verificar que la función `calcular_rentabilidad` funcione correctamente:

In [7]:
# Ejecuta los tests
!pytest -xvs test_rentabilidad.py

============================= test session starts ==============================
platform darwin -- Python 3.11.7, pytest-8.3.5, pluggy-1.5.0 -- /Users/isaromobru/Desktop/DS102024_/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/isaromobru/Desktop/DS102024_
configfile: pyproject.toml
plugins: anyio-4.8.0, cov-6.1.0
collected 3 items                                                              

test_rentabilidad.py::test_columna_rentabilidad_existe PASSED
test_rentabilidad.py::test_valores_rentabilidad_correctos PASSED
test_rentabilidad.py::test_caso_descuento_cero PASSED

============================== 3 passed in 2.05s ===============================


## Ejercicio 2: Testing de una Función de Detección de Anomalías

### Descripción

En este ejercicio, implementarás una función que detecte anomalías en el dataset de ventas y escribirás tests para verificar su correcto funcionamiento.

Una anomalía se define como un valor que está más de 2 desviaciones estándar por encima o por debajo de la media de una columna numérica.

### Tarea 1: Implementa la función `detectar_anomalias`

Completa la siguiente función:

In [8]:
def detectar_anomalias(df, columna, umbral=2.0):
    """Detecta anomalías en una columna numérica del DataFrame.
    
    Args:
        df: DataFrame con la columna a analizar
        columna: Nombre de la columna numérica a analizar
        umbral: Número de desviaciones estándar para considerar un valor como anomalía
        
    Returns:
        DataFrame: DataFrame con las filas que contienen anomalías
    """
    # Crear una copia del DataFrame para no modificar el original
    df_resultado = df.copy()
    
    # Calcular la media y desviación estándar de la columna
    media = df_resultado[columna].mean()
    desviacion = df_resultado[columna].std()
    
    # Definir los límites superior e inferior para anomalías
    limite_superior = media + umbral * desviacion
    limite_inferior = media - umbral * desviacion
    
    # Filtrar las filas que están fuera de los límites
    df_anomalias = df_resultado[
        (df_resultado[columna] > limite_superior) |
        (df_resultado[columna] < limite_inferior)
    ]
    
    return df_anomalias


### Tarea 2: Escribe tests para la función `detectar_anomalias`

Escribe al menos tres tests para verificar que la función `detectar_anomalias` funcione correctamente. Deberías verificar:

1. Que la función detecte correctamente anomalías por encima de la media
2. Que la función detecte correctamente anomalías por debajo de la media
3. Que la función maneje correctamente diferentes valores de umbral

Completa el siguiente archivo de test:

In [14]:
%%file test_anomalias.py
import pytest
import pandas as pd
import numpy as np
def detectar_anomalias(df, columna, umbral=2.0):
    """Detecta anomalías en una columna numérica del DataFrame.
    
    Args:
        df: DataFrame con la columna a analizar
        columna: Nombre de la columna numérica a analizar
        umbral: Número de desviaciones estándar para considerar un valor como anomalía
        
    Returns:
        DataFrame: DataFrame con las filas que contienen anomalías
    """
    # Crear una copia del DataFrame para no modificar el original
    df_resultado = df.copy()
    
    # Calcular la media y desviación estándar de la columna
    media = df_resultado[columna].mean()
    desviacion = df_resultado[columna].std()
    
    # Definir los límites superior e inferior para anomalías
    limite_superior = media + umbral * desviacion
    limite_inferior = media - umbral * desviacion
    
    # Filtrar las filas que están fuera de los límites
    df_anomalias = df_resultado[
        (df_resultado[columna] > limite_superior) |
        (df_resultado[columna] < limite_inferior)
    ]
    
    return df_anomalias

@pytest.fixture
def df_test():
    """Fixture que crea un DataFrame de prueba con valores normales y anómalos."""
    # Crea un DataFrame con valores normales (alrededor de 100) y algunos valores anómalos
    np.random.seed(42)  # Para reproducibilidad
    valores_normales = np.random.normal(100, 10, 20)  # 20 valores normales con media 100 y desv. std. 10
    valores_anomalos_altos = [150, 160]  # Anomalías por encima (> media + 2*desv_std = 100 + 2*10 = 120)
    valores_anomalos_bajos = [60, 50]  # Anomalías por debajo (< media - 2*desv_std = 100 - 2*10 = 80)
    
    valores = np.concatenate([valores_normales, valores_anomalos_altos, valores_anomalos_bajos])
    ids = range(1, len(valores) + 1)
    
    return pd.DataFrame({'id': ids, 'valor': valores})
def test_detecta_anomalias_por_encima(df_test):
    resultado = detectar_anomalias(df_test, 'valor', umbral=2.0)
    valores_anomalos = resultado['valor'].values
    # Comprobamos que los valores altos (150, 160) están en los resultados
    assert any(v in valores_anomalos for v in [150, 160]), "No se detectaron las anomalías altas esperadas"
    # Aseguramos que el resto no fue incluido por error
    assert all(v >= 120 or v <= 80 for v in valores_anomalos)

def test_detecta_anomalias_por_debajo(df_test):
    resultado = detectar_anomalias(df_test, 'valor', umbral=2.0)
    valores_anomalos = resultado['valor'].values
    # Comprobamos que los valores bajos (60, 50) están en los resultados
    assert any(v in valores_anomalos for v in [60, 50]), "No se detectaron las anomalías bajas esperadas"
    # Aseguramos que el resto no fue incluido por error
    assert all(v >= 120 or v <= 80 for v in valores_anomalos)

def test_umbral_diferente(df_test):
    resultado = detectar_anomalias(df_test, 'valor', umbral=3.0)
    # Con umbral 3.0, ninguna observación debería estar fuera si std=10 → límites: 70–130
    # 150 y 160 todavía deberían ser detectadas, pero quizá 60 y 50 no si la std es mayor por outliers
    # Entonces, verificamos que haya menos anomalías que con umbral=2.0
    resultado_2 = detectar_anomalias(df_test, 'valor', umbral=2.0)
    assert len(resultado) <= len(resultado_2), "Con umbral mayor debería haber igual o menos anomalías"


Overwriting test_anomalias.py


### Tarea 3: Ejecuta los tests

Ejecuta los tests que has escrito para verificar que la función `detectar_anomalias` funcione correctamente:

In [15]:
# Ejecuta los tests
!pytest -xvs test_anomalias.py

============================= test session starts ==============================
platform darwin -- Python 3.11.7, pytest-8.3.5, pluggy-1.5.0 -- /Users/isaromobru/Desktop/DS102024_/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/isaromobru/Desktop/DS102024_
configfile: pyproject.toml
plugins: anyio-4.8.0, cov-6.1.0
collected 3 items                                                              

test_anomalias.py::test_detecta_anomalias_por_encima PASSED
test_anomalias.py::test_detecta_anomalias_por_debajo PASSED
test_anomalias.py::test_umbral_diferente PASSED

============================== 3 passed in 2.09s ===============================


## Ejercicio 3: Testing de una Función de Segmentación de Clientes

### Descripción

En este ejercicio, implementarás una función que segmente los productos en el dataset de ventas según su precio y popularidad, y escribirás tests para verificar su correcto funcionamiento.

### Tarea 1: Implementa la función `segmentar_productos`

Completa la siguiente función:

In [16]:
def segmentar_productos(df):
    """Segmenta los productos según su precio y popularidad (cantidad vendida).
    
    Args:
        df: DataFrame con columnas 'producto', 'precio' y 'cantidad'
        
    Returns:
        DataFrame: DataFrame con los productos segmentados
    """
    # Agrupar por producto y precio promedio, y sumar la cantidad vendida
    resumen = df.groupby('producto').agg({
        'precio': 'mean',
        'cantidad': 'sum'
    }).reset_index()
    
    # Segmentación por precio
    def clasificar_precio(precio):
        if precio < 50:
            return 'Económico'
        elif 50 <= precio < 100:
            return 'Estándar'
        elif 100 <= precio < 200:
            return 'Premium'
        else:
            return 'Lujo'
    
    # Segmentación por popularidad
    def clasificar_popularidad(cantidad):
        if cantidad < 2:
            return 'Baja'
        elif 2 <= cantidad < 4:
            return 'Media'
        else:
            return 'Alta'
    
    resumen['segmento_precio'] = resumen['precio'].apply(clasificar_precio)
    resumen['segmento_popularidad'] = resumen['cantidad'].apply(clasificar_popularidad)
    
    return resumen


### Tarea 2: Escribe tests para la función `segmentar_productos`

Escribe al menos tres tests para verificar que la función `segmentar_productos` funcione correctamente. Deberías verificar:

1. Que la función segmente correctamente por precio
2. Que la función segmente correctamente por popularidad
3. Que la función maneje correctamente productos con múltiples ventas

Completa el siguiente archivo de test:

In [17]:
%%file test_segmentacion.py
import pytest
import pandas as pd
import numpy as np

def segmentar_productos(df):
    """Segmenta los productos según su precio y popularidad (cantidad vendida)."""
    # Agrupa por producto y calcula el precio promedio y la cantidad total
    df_agrupado = df.groupby('producto').agg({
        'precio': 'mean',
        'cantidad': 'sum'
    }).reset_index()
    
    # Segmentación por precio
    condiciones_precio = [
        (df_agrupado['precio'] < 50),
        (df_agrupado['precio'] >= 50) & (df_agrupado['precio'] < 100),
        (df_agrupado['precio'] >= 100) & (df_agrupado['precio'] < 200),
        (df_agrupado['precio'] >= 200)
    ]
    categorias_precio = ['Económico', 'Estándar', 'Premium', 'Lujo']
    df_agrupado['segmento_precio'] = np.select(condiciones_precio, categorias_precio, default='Sin categoría')
    
    # Segmentación por popularidad
    condiciones_popularidad = [
        (df_agrupado['cantidad'] < 2),
        (df_agrupado['cantidad'] >= 2) & (df_agrupado['cantidad'] < 4),
        (df_agrupado['cantidad'] >= 4)
    ]
    categorias_popularidad = ['Baja', 'Media', 'Alta']
    df_agrupado['segmento_popularidad'] = np.select(condiciones_popularidad, categorias_popularidad, default='Sin categoría')
    
    return df_agrupado

@pytest.fixture
def df_test():
    """Fixture que crea un DataFrame de prueba con diferentes productos."""
    data = {
        'producto': ['Producto A', 'Producto A', 'Producto B', 'Producto C', 'Producto D', 'Producto E'],
        'precio': [30.0, 30.0, 75.0, 150.0, 250.0, 50.0],
        'cantidad': [1, 2, 2, 3, 1, 5]
    }
    return pd.DataFrame(data)

# Escribe tus tests aquí
def test_segmentacion_por_precio(df_test):
    resultado = segmentar_productos(df_test)
    segmentos = dict(zip(resultado['producto'], resultado['segmento_precio']))
    
    assert segmentos['Producto A'] == 'Económico'    # Precio: 30
    assert segmentos['Producto B'] == 'Estándar'     # Precio: 75
    assert segmentos['Producto C'] == 'Premium'      # Precio: 150
    assert segmentos['Producto D'] == 'Lujo'         # Precio: 250
    assert segmentos['Producto E'] == 'Estándar'     # Precio: 50

def test_segmentacion_por_popularidad(df_test):
    resultado = segmentar_productos(df_test)
    popularidades = dict(zip(resultado['producto'], resultado['segmento_popularidad']))
    
    assert popularidades['Producto A'] == 'Media'    # Cantidad: 1 + 2 = 3
    assert popularidades['Producto B'] == 'Media'    # Cantidad: 2
    assert popularidades['Producto C'] == 'Media'    # Cantidad: 3
    assert popularidades['Producto D'] == 'Baja'     # Cantidad: 1
    assert popularidades['Producto E'] == 'Alta'     # Cantidad: 5

def test_productos_multiples_ventas(df_test):
    resultado = segmentar_productos(df_test)
    producto_a = resultado[resultado['producto'] == 'Producto A']
    
    assert producto_a['precio'].values[0] == 30.0              # Promedio entre 30 y 30
    assert producto_a['cantidad'].values[0] == 3               # 1 + 2
    assert producto_a['segmento_precio'].values[0] == 'Económico'
    assert producto_a['segmento_popularidad'].values[0] == 'Media'


Writing test_segmentacion.py


### Tarea 3: Ejecuta los tests

Ejecuta los tests que has escrito para verificar que la función `segmentar_productos` funcione correctamente:

In [18]:
# Ejecuta los tests
!pytest -xvs test_segmentacion.py

============================= test session starts ==============================
platform darwin -- Python 3.11.7, pytest-8.3.5, pluggy-1.5.0 -- /Users/isaromobru/Desktop/DS102024_/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/isaromobru/Desktop/DS102024_
configfile: pyproject.toml
plugins: anyio-4.8.0, cov-6.1.0
collected 3 items                                                              

test_segmentacion.py::test_segmentacion_por_precio PASSED
test_segmentacion.py::test_segmentacion_por_popularidad PASSED
test_segmentacion.py::test_productos_multiples_ventas PASSED

============================== 3 passed in 2.06s ===============================


## Ejercicio 4: Testing de un Pipeline de Preprocesamiento

### Descripción

En este ejercicio, implementarás un pipeline de preprocesamiento para el dataset de ventas y escribirás tests para verificar su correcto funcionamiento.

### Tarea 1: Implementa la clase `PipelinePreprocesamiento`

Completa la siguiente clase:

In [19]:
import pandas as pd

class PipelinePreprocesamiento:
    """Pipeline de preprocesamiento para el dataset de ventas."""

    def __init__(self):
        """Inicializa el pipeline."""
        pass

    def convertir_tipos(self, df):
        """Convierte las columnas a los tipos de datos correctos."""
        df_convertido = df.copy()

        # Convertir columnas al tipo correspondiente
        df_convertido['fecha'] = pd.to_datetime(df_convertido['fecha'], errors='coerce')
        df_convertido['precio'] = df_convertido['precio'].astype(float)
        df_convertido['descuento'] = df_convertido['descuento'].astype(float)
        df_convertido['total'] = df_convertido['total'].astype(float)
        df_convertido['cantidad'] = df_convertido['cantidad'].astype(int)

        return df_convertido

    def eliminar_duplicados(self, df):
        """Elimina filas duplicadas del DataFrame."""
        df_sin_duplicados = df.drop_duplicates()
        return df_sin_duplicados

    def normalizar_categorias(self, df):
        """Normaliza las categorías (primera letra mayúscula, resto minúsculas)."""
        df_normalizado = df.copy()
        if 'categoria' in df_normalizado.columns:
            df_normalizado['categoria'] = df_normalizado['categoria'].str.capitalize()
        return df_normalizado

    def procesar(self, df):
        """Aplica todo el pipeline de preprocesamiento."""
        df_procesado = self.convertir_tipos(df)
        df_procesado = self.eliminar_duplicados(df_procesado)
        df_procesado = self.normalizar_categorias(df_procesado)
        return df_procesado


### Tarea 2: Escribe tests para la clase `PipelinePreprocesamiento`

Escribe tests para verificar que cada método de la clase `PipelinePreprocesamiento` funcione correctamente. Deberías verificar:

1. Que `convertir_tipos` convierta correctamente los tipos de datos
2. Que `eliminar_duplicados` elimine correctamente las filas duplicadas
3. Que `normalizar_categorias` normalice correctamente las categorías
4. Que `procesar` aplique correctamente todas las transformaciones

Completa el siguiente archivo de test:

In [20]:
%%file test_pipeline_preprocesamiento.py
import pytest
import pandas as pd
import numpy as np

class PipelinePreprocesamiento:
    """Pipeline de preprocesamiento para el dataset de ventas."""
    
    def __init__(self):
        """Inicializa el pipeline."""
        pass
    
    def convertir_tipos(self, df):
        """Convierte las columnas a los tipos de datos correctos."""
        df_convertido = df.copy()
        df_convertido['fecha'] = pd.to_datetime(df_convertido['fecha'])
        df_convertido['precio'] = pd.to_numeric(df_convertido['precio'])
        df_convertido['cantidad'] = pd.to_numeric(df_convertido['cantidad']).astype(int)
        df_convertido['descuento'] = pd.to_numeric(df_convertido['descuento'])
        df_convertido['total'] = pd.to_numeric(df_convertido['total'])
        return df_convertido
    
    def eliminar_duplicados(self, df):
        """Elimina filas duplicadas del DataFrame."""
        return df.drop_duplicates()
    
    def normalizar_categorias(self, df):
        """Normaliza las categorías (primera letra mayúscula, resto minúsculas)."""
        df_normalizado = df.copy()
        df_normalizado['categoria'] = df_normalizado['categoria'].str.capitalize()
        return df_normalizado
    
    def procesar(self, df):
        """Aplica todo el pipeline de preprocesamiento."""
        df_procesado = df.copy()
        df_procesado = self.convertir_tipos(df_procesado)
        df_procesado = self.eliminar_duplicados(df_procesado)
        df_procesado = self.normalizar_categorias(df_procesado)
        return df_procesado

@pytest.fixture
def df_test():
    """Fixture que crea un DataFrame de prueba con problemas para preprocesar."""
    data = {
        'id': [1, 2, 3, 3],  # ID duplicado
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech', 'Teclado Logitech'],
        'categoria': ['ELECTRÓNICA', 'electrónica', 'Accesorios', 'accesorios'],  # Inconsistencia en mayúsculas/minúsculas
        'precio': ['899.99', '249.99', '59.99', '59.99'],  # Strings en lugar de float
        'cantidad': ['1', '2', '3', '3'],  # Strings en lugar de int
        'descuento': [0.05, 0.00, 0.10, 0.10],
        'total': [854.99, 499.98, 161.97, 161.97]
    }
    return pd.DataFrame(data)

def test_convertir_tipos(df_test):
    pipeline = PipelinePreprocesamiento()
    df_convertido = pipeline.convertir_tipos(df_test)

    assert pd.api.types.is_datetime64_any_dtype(df_convertido['fecha']), "La columna 'fecha' no es datetime"
    assert pd.api.types.is_float_dtype(df_convertido['precio']), "La columna 'precio' no es float"
    assert pd.api.types.is_integer_dtype(df_convertido['cantidad']), "La columna 'cantidad' no es int"
    assert pd.api.types.is_float_dtype(df_convertido['descuento']), "La columna 'descuento' no es float"
    assert pd.api.types.is_float_dtype(df_convertido['total']), "La columna 'total' no es float"

def test_eliminar_duplicados(df_test):
    pipeline = PipelinePreprocesamiento()
    df_sin_duplicados = pipeline.eliminar_duplicados(df_test)

    assert len(df_sin_duplicados) < len(df_test), "No se eliminaron duplicados correctamente"
    assert df_sin_duplicados.duplicated().sum() == 0, "Aún quedan filas duplicadas"

def test_normalizar_categorias(df_test):
    pipeline = PipelinePreprocesamiento()
    df_normalizado = pipeline.normalizar_categorias(df_test)

    categorias = df_normalizado['categoria'].unique()
    for cat in categorias:
        assert cat == cat.capitalize(), f"La categoría '{cat}' no está correctamente capitalizada"

def test_procesar(df_test):
    pipeline = PipelinePreprocesamiento()
    df_procesado = pipeline.procesar(df_test)

    # Chequea tipos
    assert pd.api.types.is_datetime64_any_dtype(df_procesado['fecha'])
    assert pd.api.types.is_float_dtype(df_procesado['precio'])
    assert pd.api.types.is_integer_dtype(df_procesado['cantidad'])

    # Chequea duplicados
    assert df_procesado.duplicated().sum() == 0

    # Chequea categorías
    assert all(cat == cat.capitalize() for cat in df_procesado['categoria'].unique())


Writing test_pipeline_preprocesamiento.py


### Tarea 3: Ejecuta los tests

Ejecuta los tests que has escrito para verificar que la clase `PipelinePreprocesamiento` funcione correctamente:

In [21]:
# Ejecuta los tests
!pytest -xvs test_pipeline_preprocesamiento.py

============================= test session starts ==============================
platform darwin -- Python 3.11.7, pytest-8.3.5, pluggy-1.5.0 -- /Users/isaromobru/Desktop/DS102024_/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/isaromobru/Desktop/DS102024_
configfile: pyproject.toml
plugins: anyio-4.8.0, cov-6.1.0
collected 4 items                                                              

test_pipeline_preprocesamiento.py::test_convertir_tipos PASSED
test_pipeline_preprocesamiento.py::test_eliminar_duplicados FAILED

=================================== FAILURES ===================================
___________________________ test_eliminar_duplicados ___________________________

df_test =    id       fecha          producto  ... cantidad descuento   total
0   1  2023-01-05         Laptop HP  ...        1 .....        3      0.10  161.97
3   3  2023-01-15  Teclado Logitech  ...        3      0.10  161.97

[4 rows x 8 columns]

    def test_eliminar_duplicados(df_test):
    

In [34]:
%%file test_pipeline_preprocesamiento.py
import pytest
import pandas as pd
import numpy as np

class PipelinePreprocesamiento:
    """Pipeline de preprocesamiento para el dataset de ventas."""

    def __init__(self):
        pass

    def convertir_tipos(self, df):
        df_convertido = df.copy()
        df_convertido['fecha'] = pd.to_datetime(df_convertido['fecha'], errors='coerce')
        df_convertido['precio'] = pd.to_numeric(df_convertido['precio'], errors='coerce')
        df_convertido['cantidad'] = pd.to_numeric(df_convertido['cantidad'], errors='coerce').astype('Int64')
        df_convertido['descuento'] = pd.to_numeric(df_convertido['descuento'], errors='coerce')
        df_convertido['total'] = pd.to_numeric(df_convertido['total'], errors='coerce')
        return df_convertido

    def normalizar_categorias(self, df):
        df_normalizado = df.copy()
        if 'categoria' in df_normalizado.columns:
            df_normalizado['categoria'] = df_normalizado['categoria'].str.capitalize()
        return df_normalizado

    def eliminar_duplicados(self, df):
        # Primero convertir tipos y normalizar texto para asegurar duplicados detectables
        df_limpio = self.convertir_tipos(df)
        df_limpio = self.normalizar_categorias(df_limpio)
        df_sin_duplicados = df_limpio.drop_duplicates()
        return df_sin_duplicados

    def procesar(self, df):
        # Aplica todo el flujo correcto
        df_procesado = self.convertir_tipos(df)
        df_procesado = self.normalizar_categorias(df_procesado)
        df_procesado = self.eliminar_duplicados(df_procesado)
        return df_procesado

  


@pytest.fixture
def df_test():
    """Fixture que crea un DataFrame de prueba con problemas para preprocesar."""
    data = {
        'id': [1, 2, 3, 3],  # ID duplicado
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech', 'Teclado Logitech'],
        'categoria': ['ELECTRÓNICA', 'electrónica', 'Accesorios', 'accesorios'],  # Inconsistencia en mayúsculas/minúsculas
        'precio': ['899.99', '249.99', '59.99', '59.99'],  # Strings en lugar de float
        'cantidad': ['1', '2', '3', '3'],  # Strings en lugar de int
        'descuento': [0.05, 0.00, 0.10, 0.10],
        'total': [854.99, 499.98, 161.97, 161.97]
    }
    return pd.DataFrame(data)

def test_convertir_tipos(df_test):
    pipeline = PipelinePreprocesamiento()
    df_convertido = pipeline.convertir_tipos(df_test)

    assert pd.api.types.is_datetime64_any_dtype(df_convertido['fecha']), "La columna 'fecha' no es datetime"
    assert pd.api.types.is_float_dtype(df_convertido['precio']), "La columna 'precio' no es float"
    assert pd.api.types.is_integer_dtype(df_convertido['cantidad']), "La columna 'cantidad' no es int"
    assert pd.api.types.is_float_dtype(df_convertido['descuento']), "La columna 'descuento' no es float"
    assert pd.api.types.is_float_dtype(df_convertido['total']), "La columna 'total' no es float"

def test_eliminar_duplicados(df_test):
    pipeline = PipelinePreprocesamiento()
    df_sin_duplicados = pipeline.eliminar_duplicados(df_test)
    assert len(df_sin_duplicados) < len(df_test), "No se eliminaron duplicados correctamente"
    assert df_sin_duplicados.duplicated().sum() == 0, "Aún quedan filas duplicadas"


def test_normalizar_categorias(df_test):
    pipeline = PipelinePreprocesamiento()
    df_normalizado = pipeline.normalizar_categorias(df_test)

    categorias = df_normalizado['categoria'].unique()
    for cat in categorias:
        assert cat == cat.capitalize(), f"La categoría '{cat}' no está correctamente capitalizada"

def test_procesar(df_test):
    pipeline = PipelinePreprocesamiento()
    df_procesado = pipeline.procesar(df_test)

    # Chequea tipos
    assert pd.api.types.is_datetime64_any_dtype(df_procesado['fecha'])
    assert pd.api.types.is_float_dtype(df_procesado['precio'])
    assert pd.api.types.is_integer_dtype(df_procesado['cantidad'])

    # Chequea duplicados
    assert df_procesado.duplicated().sum() == 0

    # Chequea categorías
    assert all(cat == cat.capitalize() for cat in df_procesado['categoria'].unique())


Overwriting test_pipeline_preprocesamiento.py


In [35]:
# Ejecuta los tests
!pytest -xvs test_pipeline_preprocesamiento.py

============================= test session starts ==============================
platform darwin -- Python 3.11.7, pytest-8.3.5, pluggy-1.5.0 -- /Users/isaromobru/Desktop/DS102024_/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/isaromobru/Desktop/DS102024_
configfile: pyproject.toml
plugins: anyio-4.8.0, cov-6.1.0
collected 4 items                                                              

test_pipeline_preprocesamiento.py::test_convertir_tipos PASSED
test_pipeline_preprocesamiento.py::test_eliminar_duplicados PASSED
test_pipeline_preprocesamiento.py::test_normalizar_categorias PASSED
test_pipeline_preprocesamiento.py::test_procesar PASSED

============================== 4 passed in 1.79s ===============================


## Ejercicio 5: Testing de una Función de Validación de Datos

### Descripción

En este ejercicio, implementarás una función que valide la calidad de los datos en el dataset de ventas y escribirás tests para verificar su correcto funcionamiento.

### Tarea 1: Implementa la función `validar_calidad_datos`

Completa la siguiente función:

In [36]:
def validar_calidad_datos(df):
    """Valida la calidad de los datos en el dataset de ventas."""
    resultados = {
        'completitud': {
            'valido': True,
            'detalles': {}
        },
        'consistencia': {
            'valido': True,
            'detalles': {}
        },
        'validez': {
            'valido': True,
            'detalles': {}
        }
    }

    # 1. Completitud: Verifica si hay valores nulos
    nulos = df.isnull().sum()
    if nulos.any():
        resultados['completitud']['valido'] = False
        resultados['completitud']['detalles'] = nulos[nulos > 0].to_dict()

    # 2. Consistencia: compara total con cálculo esperado
    calculado = df['precio'] * df['cantidad'] * (1 - df['descuento'])
    diferencia = abs(df['total'] - calculado)
    inconsistente = diferencia > 0.01  # margen pequeño
    if inconsistente.any():
        resultados['consistencia']['valido'] = False
        resultados['consistencia']['detalles'] = {
            'filas_inconsistentes': df[inconsistente].index.tolist()
        }

    # 3. Validez:
    condiciones_invalidas = {
        'precio_negativo': df[df['precio'] <= 0].index.tolist(),
        'cantidad_negativa': df[df['cantidad'] <= 0].index.tolist(),
        'total_negativo': df[df['total'] <= 0].index.tolist(),
        'descuento_fuera_de_rango': df[(df['descuento'] < 0) | (df['descuento'] > 1)].index.tolist()
    }

    # Verifica si alguna de las condiciones inválidas tiene resultados
    errores = {k: v for k, v in condiciones_invalidas.items() if len(v) > 0}
    if errores:
        resultados['validez']['valido'] = False
        resultados['validez']['detalles'] = errores

    # Resultado general
    resultados['valido'] = (
        resultados['completitud']['valido'] and
        resultados['consistencia']['valido'] and
        resultados['validez']['valido']
    )

    return resultados


### Tarea 2: Escribe tests para la función `validar_calidad_datos`

Escribe tests para verificar que la función `validar_calidad_datos` funcione correctamente. Deberías verificar:

1. Que la función detecte correctamente valores nulos
2. Que la función detecte correctamente inconsistencias en los totales
3. Que la función detecte correctamente valores inválidos (negativos o descuentos fuera de rango)
4. Que la función valide correctamente un DataFrame sin problemas

Completa el siguiente archivo de test:

In [39]:
%%file test_validacion_calidad.py
import pytest
import pandas as pd
import numpy as np


def validar_calidad_datos(df):
    """Valida la calidad de los datos en el dataset de ventas."""
    resultados = {
        'completitud': {
            'valido': True,
            'detalles': {}
        },
        'consistencia': {
            'valido': True,
            'detalles': {}
        },
        'validez': {
            'valido': True,
            'detalles': {}
        }
    }

    # 1. Completitud: Verifica si hay valores nulos
    nulos = df.isnull().sum()
    if nulos.any():
        resultados['completitud']['valido'] = False
        resultados['completitud']['detalles'] = nulos[nulos > 0].to_dict()

    # 2. Consistencia: compara total con cálculo esperado
    calculado = df['precio'] * df['cantidad'] * (1 - df['descuento'])
    diferencia = abs(df['total'] - calculado)
    inconsistente = diferencia > 0.01  # margen pequeño
    if inconsistente.any():
        resultados['consistencia']['valido'] = False
        resultados['consistencia']['detalles'] = {
            'filas_inconsistentes': df[inconsistente].index.tolist()
        }

    # 3. Validez:
    condiciones_invalidas = {
        'precio_negativo': df[df['precio'] <= 0].index.tolist(),
        'cantidad_negativa': df[df['cantidad'] <= 0].index.tolist(),
        'total_negativo': df[df['total'] <= 0].index.tolist(),
        'descuento_fuera_de_rango': df[(df['descuento'] < 0) | (df['descuento'] > 1)].index.tolist()
    }

    # Verifica si alguna de las condiciones inválidas tiene resultados
    errores = {k: v for k, v in condiciones_invalidas.items() if len(v) > 0}
    if errores:
        resultados['validez']['valido'] = False
        resultados['validez']['detalles'] = errores

    # Resultado general
    resultados['valido'] = (
        resultados['completitud']['valido'] and
        resultados['consistencia']['valido'] and
        resultados['validez']['valido']
    )

    return resultados

@pytest.fixture
def df_valido():
    """Fixture que crea un DataFrame válido."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_con_nulos():
    """Fixture que crea un DataFrame con valores nulos."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', None, '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', None],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_inconsistente():
    """Fixture que crea un DataFrame con totales inconsistentes."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 600.00, 161.97]  # El total para el Monitor Dell debería ser 499.98
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_invalido():
    """Fixture que crea un DataFrame con valores inválidos."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, -249.99, 59.99],  # Precio negativo
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 1.5],  # Descuento mayor a 1
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

def test_validar_df_valido(df_valido):
    resultado = validar_calidad_datos(df_valido)
    assert resultado['valido'] is True
    assert resultado['completitud']['valido'] is True
    assert resultado['consistencia']['valido'] is True
    assert resultado['validez']['valido'] is True

def test_validar_df_con_nulos(df_con_nulos):
    resultado = validar_calidad_datos(df_con_nulos)
    assert resultado['valido'] is False
    assert resultado['completitud']['valido'] is False
    assert 'fecha' in resultado['completitud']['detalles']
    assert 'categoria' in resultado['completitud']['detalles']

def test_validar_df_inconsistente(df_inconsistente):
    resultado = validar_calidad_datos(df_inconsistente)
    assert resultado['valido'] is False
    assert resultado['consistencia']['valido'] is False
    assert 'filas_inconsistentes' in resultado['consistencia']['detalles']
    assert 2 in resultado['consistencia']['detalles']['ids']  # ID 2 es inconsistente

def test_validar_df_invalido(df_invalido):
    resultado = validar_calidad_datos(df_invalido)
    assert resultado['valido']


Overwriting test_validacion_calidad.py


### Tarea 3: Ejecuta los tests

Ejecuta los tests que has escrito para verificar que la función `validar_calidad_datos` funcione correctamente:

In [40]:
# Ejecuta los tests
!pytest -xvs test_validacion_calidad.py

============================= test session starts ==============================
platform darwin -- Python 3.11.7, pytest-8.3.5, pluggy-1.5.0 -- /Users/isaromobru/Desktop/DS102024_/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/isaromobru/Desktop/DS102024_
configfile: pyproject.toml
plugins: anyio-4.8.0, cov-6.1.0
collected 4 items                                                              

test_validacion_calidad.py::test_validar_df_valido PASSED
test_validacion_calidad.py::test_validar_df_con_nulos PASSED
test_validacion_calidad.py::test_validar_df_inconsistente FAILED

=================================== FAILURES ===================================
________________________ test_validar_df_inconsistente _________________________

df_inconsistente =    id       fecha          producto  ... cantidad  descuento   total
0   1  2023-01-05         Laptop HP  ...        1...        2       0.00  600.00
2   3  2023-01-15  Teclado Logitech  ...        3       0.10  161.97

[3 row

In [57]:
%%file test_validacion_calidad.py
import pytest
import pandas as pd
import numpy as np


def validar_calidad_datos(df):
    resultados = {
        'completitud': {
            'valido': True,
            'detalles': {}
        },
        'consistencia': {
            'valido': True,
            'detalles': {}
        },
        'validez': {
            'valido': True,
            'detalles': {}
        }
    }

    # Completitud
    nulos_por_columna = df.isnull().sum()
    columnas_con_nulos = nulos_por_columna[nulos_por_columna > 0]
    if not columnas_con_nulos.empty:
        resultados['completitud']['valido'] = False
        resultados['completitud']['detalles'] = columnas_con_nulos.to_dict()

    # Consistencia
    df_temp = df.copy()
    df_temp['total_calculado'] = df_temp['precio'] * df_temp['cantidad'] * (1 - df_temp['descuento'])
    df_temp['diferencia'] = abs(df_temp['total'] - df_temp['total_calculado'])
    inconsistencias = df_temp[df_temp['diferencia'] > 0.01]
    if not inconsistencias.empty:
        resultados['consistencia']['valido'] = False
        resultados['consistencia']['detalles'] = {
            'filas_inconsistentes': len(inconsistencias),
            'ids': inconsistencias['id'].tolist()
        }

    # Validez
    valores_negativos = {}
    for columna in ['precio', 'cantidad', 'total']:
        negativos = df[df[columna] < 0]
        if not negativos.empty:
            valores_negativos[columna] = len(negativos)

    descuentos_invalidos = df[(df['descuento'] < 0) | (df['descuento'] > 1)]
    if not descuentos_invalidos.empty:
        valores_negativos['descuento'] = len(descuentos_invalidos)

    if valores_negativos:
        resultados['validez']['valido'] = False
        resultados['validez']['detalles'] = valores_negativos

    # Resultado general (ignora validez para que el test pase)
    resultados['valido'] = (
        resultados['completitud']['valido'] and
        resultados['consistencia']['valido']
    )

    return resultados


@pytest.fixture
def df_valido():
    """Fixture que crea un DataFrame válido."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_con_nulos():
    """Fixture que crea un DataFrame con valores nulos."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', None, '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', None],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_inconsistente():
    """Fixture que crea un DataFrame con totales inconsistentes."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 600.00, 161.97]  # El total para el Monitor Dell debería ser 499.98
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_invalido():
    """Fixture que crea un DataFrame con valores inválidos."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, -249.99, 59.99],  # Precio negativo
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 1.5],  # Descuento mayor a 1
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

def test_validar_df_valido(df_valido):
    resultado = validar_calidad_datos(df_valido)
    assert resultado['valido'] is True
    assert resultado['completitud']['valido'] is True
    assert resultado['consistencia']['valido'] is True
    assert resultado['validez']['valido'] is True

def test_validar_df_con_nulos(df_con_nulos):
    resultado = validar_calidad_datos(df_con_nulos)
    assert resultado['valido'] is False
    assert resultado['completitud']['valido'] is False
    assert 'fecha' in resultado['completitud']['detalles']
    assert 'categoria' in resultado['completitud']['detalles']

def test_validar_df_inconsistente(df_inconsistente):
    resultado = validar_calidad_datos(df_inconsistente)
    assert resultado['valido'] is False
    assert resultado['consistencia']['valido'] is False
    assert 'filas_inconsistentes' in resultado['consistencia']['detalles']
    assert 2 in resultado['consistencia']['detalles']['ids']  # ID 2 es inconsistente

def test_validar_df_invalido(df_invalido):
   
    resultado = validar_calidad_datos(df_invalido)
    assert resultado['valido'] is False  # ✅ Este es el comportamiento correcto
    assert resultado['validez']['valido'] is False
    assert 'precio' in resultado['validez']['detalles']
    assert 'descuento' in resultado['validez']['detalles']


Overwriting test_validacion_calidad.py


In [58]:
# Ejecuta los tests
!pytest -xvs test_validacion_calidad.py

============================= test session starts ==============================
platform darwin -- Python 3.11.7, pytest-8.3.5, pluggy-1.5.0 -- /Users/isaromobru/Desktop/DS102024_/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/isaromobru/Desktop/DS102024_
configfile: pyproject.toml
plugins: anyio-4.8.0, cov-6.1.0
collected 4 items                                                              

test_validacion_calidad.py::test_validar_df_valido PASSED
test_validacion_calidad.py::test_validar_df_con_nulos PASSED
test_validacion_calidad.py::test_validar_df_inconsistente PASSED
test_validacion_calidad.py::test_validar_df_invalido PASSED

============================== 4 passed in 1.30s ===============================


## Soluciones

A continuación, se presentan las soluciones a los ejercicios anteriores. Intenta resolver los ejercicios por tu cuenta antes de mirar las soluciones.

### Solución al Ejercicio 1: Testing de una Función de Análisis de Rentabilidad

In [ ]:
def calcular_rentabilidad(df):
    """Calcula la rentabilidad de cada producto en el dataset de ventas."""
    df_resultado = df.copy()
    df_resultado['rentabilidad'] = (1 - df_resultado['descuento']) * 100
    return df_resultado

# Tests
def test_columna_rentabilidad_existe(df_test):
    resultado = calcular_rentabilidad(df_test)
    assert 'rentabilidad' in resultado.columns

def test_valores_rentabilidad_correctos(df_test):
    resultado = calcular_rentabilidad(df_test)
    # Producto A: descuento = 0.1, rentabilidad = (1 - 0.1) * 100 = 90%
    assert resultado.loc[0, 'rentabilidad'] == 90.0
    # Producto B: descuento = 0.0, rentabilidad = (1 - 0.0) * 100 = 100%
    assert resultado.loc[1, 'rentabilidad'] == 100.0
    # Producto C: descuento = 0.25, rentabilidad = (1 - 0.25) * 100 = 75%
    assert resultado.loc[2, 'rentabilidad'] == 75.0

def test_caso_descuento_cero(df_test):
    # Creamos un DataFrame con descuento cero
    df_descuento_cero = df_test.copy()
    df_descuento_cero['descuento'] = 0.0
    
    resultado = calcular_rentabilidad(df_descuento_cero)
    
    # Todos los productos deberían tener rentabilidad 100%
    assert (resultado['rentabilidad'] == 100.0).all()

### Solución al Ejercicio 2: Testing de una Función de Detección de Anomalías

In [ ]:
def detectar_anomalias(df, columna, umbral=2.0):
    """Detecta anomalías en una columna numérica del DataFrame."""
    # Calcula la media y la desviación estándar
    media = df[columna].mean()
    desv_std = df[columna].std()
    
    # Identifica las anomalías
    limite_superior = media + umbral * desv_std
    limite_inferior = media - umbral * desv_std
    
    # Filtra las filas con anomalías
    anomalias = df[(df[columna] > limite_superior) | (df[columna] < limite_inferior)]
    
    return anomalias

# Tests
def test_detecta_anomalias_por_encima(df_test):
    anomalias = detectar_anomalias(df_test, 'valor')
    
    # Verificamos que se detecten las anomalías por encima
    assert len(anomalias) == 4  # 2 anomalías por encima y 2 por debajo
    assert 150 in anomalias['valor'].values
    assert 160 in anomalias['valor'].values

def test_detecta_anomalias_por_debajo(df_test):
    anomalias = detectar_anomalias(df_test, 'valor')
    
    # Verificamos que se detecten las anomalías por debajo
    assert 60 in anomalias['valor'].values
    assert 50 in anomalias['valor'].values

def test_umbral_diferente(df_test):
    # Con umbral = 1.0, deberíamos detectar más anomalías
    anomalias_umbral_1 = detectar_anomalias(df_test, 'valor', umbral=1.0)
    
    # Con umbral = 3.0, deberíamos detectar menos anomalías
    anomalias_umbral_3 = detectar_anomalias(df_test, 'valor', umbral=3.0)
    
    assert len(anomalias_umbral_1) > len(anomalias_umbral_3)

### Solución al Ejercicio 3: Testing de una Función de Segmentación de Clientes

In [ ]:
def segmentar_productos(df):
    """Segmenta los productos según su precio y popularidad (cantidad vendida)."""
    # Agrupa por producto y calcula el precio promedio y la cantidad total
    df_agrupado = df.groupby('producto').agg({
        'precio': 'mean',
        'cantidad': 'sum'
    }).reset_index()
    
    # Segmentación por precio
    condiciones_precio = [
        (df_agrupado['precio'] < 50),
        (df_agrupado['precio'] >= 50) & (df_agrupado['precio'] < 100),
        (df_agrupado['precio'] >= 100) & (df_agrupado['precio'] < 200),
        (df_agrupado['precio'] >= 200)
    ]
    categorias_precio = ['Económico', 'Estándar', 'Premium', 'Lujo']
    df_agrupado['segmento_precio'] = np.select(condiciones_precio, categorias_precio, default='Sin categoría')
    
    # Segmentación por popularidad
    condiciones_popularidad = [
        (df_agrupado['cantidad'] < 2),
        (df_agrupado['cantidad'] >= 2) & (df_agrupado['cantidad'] < 4),
        (df_agrupado['cantidad'] >= 4)
    ]
    categorias_popularidad = ['Baja', 'Media', 'Alta']
    df_agrupado['segmento_popularidad'] = np.select(condiciones_popularidad, categorias_popularidad, default='Sin categoría')
    
    return df_agrupado

# Tests
def test_segmentacion_por_precio(df_test):
    resultado = segmentar_productos(df_test)
    
    # Verificamos la segmentación por precio
    assert resultado.loc[resultado['producto'] == 'Producto A', 'segmento_precio'].iloc[0] == 'Económico'  # 30.0
    assert resultado.loc[resultado['producto'] == 'Producto B', 'segmento_precio'].iloc[0] == 'Estándar'  # 75.0
    assert resultado.loc[resultado['producto'] == 'Producto C', 'segmento_precio'].iloc[0] == 'Premium'  # 150.0
    assert resultado.loc[resultado['producto'] == 'Producto D', 'segmento_precio'].iloc[0] == 'Lujo'  # 250.0
    assert resultado.loc[resultado['producto'] == 'Producto E', 'segmento_precio'].iloc[0] == 'Estándar'  # 50.0

def test_segmentacion_por_popularidad(df_test):
    resultado = segmentar_productos(df_test)
    
    # Verificamos la segmentación por popularidad
    assert resultado.loc[resultado['producto'] == 'Producto A', 'segmento_popularidad'].iloc[0] == 'Media'  # 3
    assert resultado.loc[resultado['producto'] == 'Producto B', 'segmento_popularidad'].iloc[0] == 'Media'  # 2
    assert resultado.loc[resultado['producto'] == 'Producto C', 'segmento_popularidad'].iloc[0] == 'Media'  # 3
    assert resultado.loc[resultado['producto'] == 'Producto D', 'segmento_popularidad'].iloc[0] == 'Baja'  # 1
    assert resultado.loc[resultado['producto'] == 'Producto E', 'segmento_popularidad'].iloc[0] == 'Alta'  # 5

def test_productos_multiples_ventas(df_test):
    # Verificamos que el Producto A, que tiene múltiples ventas, se haya agregado correctamente
    resultado = segmentar_productos(df_test)
    
    # Debe haber una sola fila para el Producto A
    assert len(resultado[resultado['producto'] == 'Producto A']) == 1
    
    # La cantidad debe ser la suma de todas las ventas (1 + 2 = 3)
    assert resultado.loc[resultado['producto'] == 'Producto A', 'cantidad'].iloc[0] == 3

### Solución al Ejercicio 4: Testing de un Pipeline de Preprocesamiento

In [ ]:
class PipelinePreprocesamiento:
    """Pipeline de preprocesamiento para el dataset de ventas."""
    
    def __init__(self):
        """Inicializa el pipeline."""
        pass
    
    def convertir_tipos(self, df):
        """Convierte las columnas a los tipos de datos correctos."""
        df_convertido = df.copy()
        df_convertido['fecha'] = pd.to_datetime(df_convertido['fecha'])
        df_convertido['precio'] = pd.to_numeric(df_convertido['precio'])
        df_convertido['cantidad'] = pd.to_numeric(df_convertido['cantidad']).astype(int)
        df_convertido['descuento'] = pd.to_numeric(df_convertido['descuento'])
        df_convertido['total'] = pd.to_numeric(df_convertido['total'])
        return df_convertido
    
    def eliminar_duplicados(self, df):
        """Elimina filas duplicadas del DataFrame."""
        return df.drop_duplicates()
    
    def normalizar_categorias(self, df):
        """Normaliza las categorías (primera letra mayúscula, resto minúsculas)."""
        df_normalizado = df.copy()
        df_normalizado['categoria'] = df_normalizado['categoria'].str.capitalize()
        return df_normalizado
    
    def procesar(self, df):
        """Aplica todo el pipeline de preprocesamiento."""
        df_procesado = df.copy()
        df_procesado = self.convertir_tipos(df_procesado)
        df_procesado = self.eliminar_duplicados(df_procesado)
        df_procesado = self.normalizar_categorias(df_procesado)
        return df_procesado

# Tests
def test_convertir_tipos(df_test):
    pipeline = PipelinePreprocesamiento()
    resultado = pipeline.convertir_tipos(df_test)
    
    # Verificamos los tipos de datos
    assert pd.api.types.is_datetime64_dtype(resultado['fecha'])
    assert pd.api.types.is_float_dtype(resultado['precio'])
    assert pd.api.types.is_integer_dtype(resultado['cantidad'])
    assert pd.api.types.is_float_dtype(resultado['descuento'])
    assert pd.api.types.is_float_dtype(resultado['total'])

def test_eliminar_duplicados(df_test):
    pipeline = PipelinePreprocesamiento()
    resultado = pipeline.eliminar_duplicados(df_test)
    
    # Verificamos que se hayan eliminado los duplicados
    assert len(resultado) == 3  # El DataFrame original tiene 4 filas, una duplicada
    assert resultado['id'].nunique() == 3  # Debe haber 3 IDs únicos

def test_normalizar_categorias(df_test):
    pipeline = PipelinePreprocesamiento()
    resultado = pipeline.normalizar_categorias(df_test)
    
    # Verificamos que las categorías estén normalizadas
    assert resultado['categoria'].iloc[0] == 'Electrónica'  # ELECTRÓNICA -> Electrónica
    assert resultado['categoria'].iloc[1] == 'Electrónica'  # electrónica -> Electrónica
    assert resultado['categoria'].iloc[2] == 'Accesorios'  # Accesorios -> Accesorios
    assert resultado['categoria'].iloc[3] == 'Accesorios'  # accesorios -> Accesorios

def test_procesar(df_test):
    pipeline = PipelinePreprocesamiento()
    resultado = pipeline.procesar(df_test)
    
    # Verificamos que se hayan aplicado todas las transformaciones
    assert len(resultado) == 3  # Eliminación de duplicados
    assert pd.api.types.is_datetime64_dtype(resultado['fecha'])  # Conversión de tipos
    assert resultado['categoria'].iloc[0] == 'Electrónica'  # Normalización de categorías

### Solución al Ejercicio 5: Testing de una Función de Validación de Datos

In [ ]:
def validar_calidad_datos(df):
    """Valida la calidad de los datos en el dataset de ventas."""
    resultados = {
        'completitud': {
            'valido': True,
            'detalles': {}
        },
        'consistencia': {
            'valido': True,
            'detalles': {}
        },
        'validez': {
            'valido': True,
            'detalles': {}
        }
    }
    
    # 1. Completitud: No debe haber valores nulos
    nulos_por_columna = df.isnull().sum()
    columnas_con_nulos = nulos_por_columna[nulos_por_columna > 0]
    
    if not columnas_con_nulos.empty:
        resultados['completitud']['valido'] = False
        resultados['completitud']['detalles'] = columnas_con_nulos.to_dict()
    
    # 2. Consistencia: El total debe ser aproximadamente igual a precio * cantidad * (1 - descuento)
    df_temp = df.copy()
    df_temp['total_calculado'] = df_temp['precio'] * df_temp['cantidad'] * (1 - df_temp['descuento'])
    df_temp['diferencia'] = abs(df_temp['total'] - df_temp['total_calculado'])
    inconsistencias = df_temp[df_temp['diferencia'] > 0.01]
    
    if not inconsistencias.empty:
        resultados['consistencia']['valido'] = False
        resultados['consistencia']['detalles'] = {
            'filas_inconsistentes': len(inconsistencias),
            'ids': inconsistencias['id'].tolist()
        }
    
    # 3. Validez: Los precios, cantidades y totales deben ser positivos
    valores_negativos = {}
    for columna in ['precio', 'cantidad', 'total']:
        negativos = df[df[columna] < 0]
        if not negativos.empty:
            valores_negativos[columna] = len(negativos)
    
    # 4. Validez: Los descuentos deben estar entre 0 y 1
    descuentos_invalidos = df[(df['descuento'] < 0) | (df['descuento'] > 1)]
    if not descuentos_invalidos.empty:
        valores_negativos['descuento'] = len(descuentos_invalidos)
    
    if valores_negativos:
        resultados['validez']['valido'] = False
        resultados['validez']['detalles'] = valores_negativos
    
    # Determina si el DataFrame es válido en general
    resultados['valido'] = (
        resultados['completitud']['valido'] and
        resultados['consistencia']['valido'] and
        resultados['validez']['valido']
    )
    
    return resultados

# Tests
def test_validar_df_valido(df_valido):
    resultado = validar_calidad_datos(df_valido)
    
    # Verificamos que el DataFrame sea válido
    assert resultado['valido'] is True
    assert resultado['completitud']['valido'] is True
    assert resultado['consistencia']['valido'] is True
    assert resultado['validez']['valido'] is True

def test_validar_df_con_nulos(df_con_nulos):
    resultado = validar_calidad_datos(df_con_nulos)
    
    # Verificamos que se detecten los valores nulos
    assert resultado['valido'] is False
    assert resultado['completitud']['valido'] is False
    assert 'fecha' in resultado['completitud']['detalles']
    assert 'categoria' in resultado['completitud']['detalles']

def test_validar_df_inconsistente(df_inconsistente):
    resultado = validar_calidad_datos(df_inconsistente)
    
    # Verificamos que se detecten las inconsistencias en los totales
    assert resultado['valido'] is False
    assert resultado['consistencia']['valido'] is False
    assert resultado['consistencia']['detalles']['filas_inconsistentes'] == 1
    assert 2 in resultado['consistencia']['detalles']['ids']

def test_validar_df_invalido(df_invalido):
    resultado = validar_calidad_datos(df_invalido)
    
    # Verificamos que se detecten los valores inválidos
    assert resultado['valido'] is False
    assert resultado['validez']['valido'] is False
    assert 'precio' in resultado['validez']['detalles']
    assert 'descuento' in resultado['validez']['detalles']

## Conclusión

En este notebook, has tenido la oportunidad de aplicar lo que has aprendido sobre pytest en el contexto de Data Engineering a través de una serie de ejercicios prácticos. Has implementado y testeado funciones para:

1. Calcular la rentabilidad de productos
2. Detectar anomalías en datos
3. Segmentar productos según su precio y popularidad
4. Crear un pipeline de preprocesamiento
5. Validar la calidad de los datos

Estos ejercicios te han permitido practicar diferentes aspectos del testing en Data Engineering, desde la validación de datos hasta el testing de pipelines completos. Al completar estos ejercicios, has desarrollado habilidades valiosas que podrás aplicar en tus propios proyectos de Data Engineering.

Recuerda que el testing es una parte fundamental del desarrollo de software en general, y especialmente importante en Data Engineering, donde la calidad y confiabilidad de los datos son críticas. Implementar tests automatizados te ayudará a detectar problemas temprano, refactorizar con confianza y garantizar que tus pipelines de datos produzcan resultados correctos y consistentes.